### Собираем общий парсер

In [ ]:
# Импорт Библиотек
import re
import csv
import requests
import pandas as pd
from bs4 import BeautifulSoup
from lxml import html
from IPython.display import clear_output

In [ ]:
# Парсим ссылки на обьявления со страницы
def get_list_page_html(soup):
    list_page_html  = []
    # Парсим ссылку конкретного обьявления
    page = soup.findAll('a', class_ = 'snippet-link')

    for i in page:
        part_url = i.get('href')
        # отбор обьявлений только по нужной нам теме
        if 'iphone_7' in part_url:
            full_url = url_head + part_url
            list_page_html.append(full_url)
            print(full_url)
    return list_page_html

# Получаем цену телефона по обьявлению
def get_price (soup):
    price = int(soup.find('span', class_ = 'js-item-price').get_text().replace(' ', ''))
    #print(price)
    return price

# Получаем текст обьявления по обьявлению
def get_text (soup):
    text = soup.find('div', class_ = 'item-description-text').get_text().strip()
    #print(text)
    return text

# Получаем город обьявления по обьявлению
def get_town (soup):
    town = soup.find('span', class_ = 'item-address__string').get_text().strip()
    #print(text)
    return town

In [ ]:
# Парсим по ссылке страницы с обьявлением данные обьявления
# Ссылка на сайт который парсим
#url = list_page_html[3]
#print(url)
# Получаем HTML страницу по ссылке
def get_page_data(url):
    page = requests.get(url)
    # Получение из HTML страницы обьекта типа soup для разбора по HTML тегам
    soup = BeautifulSoup(page.text, "html.parser")
    # Код 200 означает, что сайт откликнулся и выдал нам страницу
    #print(page.status_code) 
    # Получаем город, цену и описание
    # Формат записи данных в файл
    data = {'town':get_town(soup), 'text': get_text(soup), 'price':get_price(soup)}
    print(data)
    return data

In [ ]:
list_towns = ['/moskva','/sankt-peterburg']#,'/rostov-na-donu','/krasnodar','/novosibirsk']
url_head   = 'https://www.avito.ru'
url_tail   = '/telefony/iphone-ASgBAgICAUSeAt4J?q=iphone+7&p='
file_name  = './avito.csv'

columns = ['town','text','price']
df = pd.DataFrame(columns = columns)

# Сборка полной ссылки для страницы с обьявлениями
def get_page_url(url_head, town, url_tail, num_page):
    full_url = url_head + town + url_tail + str(num_page)
     
#     print(town)
#     print(url_tail)
#     print(full_url)
    return full_url

In [ ]:
num = 0
for cur_town in list_towns:
    clear_output() # Очистка экрана    
    for num_page in range(1,3):        
        print(cur_town,' страница -', num_page)
        # Ссылка на страницу обьявления с которой парсим
        page_url = get_page_url(url_head, cur_town, url_tail, num_page)       
        print(page_url)
        try:
            # Получаем HTML страницу по ссылке
            page = requests.get(page_url)
            # Получение из HTML страницы обьекта типа soup для разбора по HTML тегам
            soup = BeautifulSoup(page.text, "html.parser")
            # Код 200 означает, что сайт откликнулся и выдал нам страницу
            print(page.status_code) 
            # Парсим ссылки на обьявления со страницы
            list_page_html = get_list_page_html(soup)
            print(list_page_html)
            for cur_page_html in list_page_html:
                try:
                    print(cur_page_html)
                    # заносим данные со страницы в датафрейм
                    df.loc[len(df)] = get_page_data(cur_page_html)
                    num += 1
                    print('Спарсено обьявлений -', num)
                except:
                    print('Ошибка')
                    pass
            # Запись данных в файл
            if len(df) > 0:
                df.to_csv(file_name, mode='a', header = columns, index=False)
                print('Сохранено успешно')
                # Сброс данных в датафрейм
            df = df.drop(df[:].index)
        except:
            #print(town,' страница -', num_page, ' не существует')
            pass
        
        
        
    